In [1]:
#Ensemble de arboles de decision
#utilizando el naif metodo de Arboles Azarosos
#entreno cada arbol utilizando un subconjunto distinto de atributos del dataset

In [2]:
#limpio la memoria
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

library("data.table")
library("rpart")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,614940,32.9,1321401,70.6,1321401,70.6
Vcells,1094727,8.4,8388608,64.0,1632451,12.5


Warning message:
"package 'data.table' was built under R version 4.2.3"
Warning message:
"package 'rpart' was built under R version 4.2.3"


In [3]:
#parmatros experimento
PARAM <- list()
PARAM$experimento  <- 3210
PARAM$semilla  <- 262139      #Establezco la semilla aleatoria, cambiar por SU primer semilla

#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  300,
                              "minbucket"=  20,
                              "maxdepth"=   10 )

#parametros  arbol
PARAM$feature_fraction  <- 0.5  #entreno cada arbol con solo 50% de las variables variables
PARAM$num_trees_max  <- 500 #voy a generar 500 arboles, a mas arboles mas tiempo de proceso y MEJOR MODELO, pero ganancias marginales

In [4]:
#Aqui comienza el programa

setwd("D:\\Onedrive\\Austral\\Cursos\\08 - Laboratorio de Implementacion 1\\labo2023v\\src\\ArbolesAzarosos")  #Establezco el Working Directory

#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")

#creo la carpeta donde va el experimento
dir.create( "./exp/", showWarnings = FALSE  )
carpeta_experimento  <-  paste0( "./exp/KA", PARAM$experimento, "/")
dir.create( paste0( "./exp/KA", PARAM$experimento, "/"), 
            showWarnings = FALSE )

setwd( carpeta_experimento )
getwd()

[1] "D:/Onedrive/Austral/Cursos/08 - Laboratorio de Implementacion 1/labo2023v/src/ArbolesAzarosos/exp/KA3210"

In [5]:
#que tamanos de ensemble grabo a disco, pero siempre debo generar los 500
grabar  <-  c( 1, 5, 10, 50, 100, 200, 500)

In [6]:
#defino los dataset de entrenamiento y aplicacion
dtrain  <- dataset[ foto_mes==202107 ]
dapply  <- dataset[ foto_mes==202109 ]

In [7]:
#aqui se va acumulando la probabilidad del ensemble
dapply[ , prob_acumulada := 0 ]

In [8]:
#Establezco cuales son los campos que puedo usar para la prediccion
#el copy() es por la Lazy Evaluation
campos_buenos  <- copy( setdiff(  colnames(dtrain) ,  c("clase_ternaria") ) )

In [9]:
#Genero las salidas
set.seed(PARAM$semilla) #Establezco la semilla aleatoria

In [10]:
# -------------------RUN 1
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  100,
                              "minbucket"=  10,
                              "maxdepth"=   6 )

corrida <- 1

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_1', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}

1  5  10  50  100  200  500  

In [11]:
# -------------------RUN 2
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  250,
                              "minbucket"=  100,
                              "maxdepth"=   14 )

corrida <- 2

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_2', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}

In [ ]:
# -------------------RUN 3
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  500,
                              "minbucket"=  200,
                              "maxdepth"=   10 )

corrida <- 3

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_3', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}

In [ ]:
# -------------------RUN 4
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  750,
                              "minbucket"=  200,
                              "maxdepth"=   6 )

corrida <- 4

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_4', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}

In [ ]:
# -------------------RUN 5
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  1000,
                              "minbucket"=  200,
                              "maxdepth"=   4 )

corrida <- 5

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_5', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}

In [ ]:
# -------------------RUN 6
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  1500,
                              "minbucket"=  200,
                              "maxdepth"=   10 )

corrida <- 6

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_6', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}

In [ ]:
# -------------------RUN 7
#parameetros rpart
PARAM$rpart_param   <- list( "cp"=          -1,
                              "minsplit"=  2000,
                              "minbucket"=  100,
                              "maxdepth"=   8 )

corrida <- 7

for( arbolito in  1:PARAM$num_trees_max )
{
  qty_campos_a_utilizar  <- as.integer( length(campos_buenos)* PARAM$feature_fraction )
  campos_random  <- sample( campos_buenos, qty_campos_a_utilizar )
  
  #paso de un vector a un string con los elementos separados por un signo de "+"
  #este hace falta para la formula
  campos_random  <- paste( campos_random, collapse=" + ")

  #armo la formula para rpart
  formulita  <- paste0( "clase_ternaria ~ ", campos_random )

  #genero el arbol de decision
  modelo  <- rpart( formulita,
                    data= dtrain,
                    xval= 0,
                    control= PARAM$rpart_param )

  #aplico el modelo a los datos que no tienen clase
  prediccion  <- predict( modelo, dapply , type = "prob")
  
  dapply[  ,  prob_acumulada :=  prob_acumulada + prediccion[ , "BAJA+2"] ]

  if( arbolito %in%  grabar )
  {
    #Genero la entrega para Kaggle
	umbral_corte  <-  (1/40) *  arbolito
    entrega  <- as.data.table( list( "numero_de_cliente"= dapply[  , numero_de_cliente],
                                     "Predicted"= as.numeric(dapply[  , prob_acumulada] > umbral_corte) ) ) #genero la salida

    nom_arch  <- paste0('KA', PARAM$experimento, "_",
						sprintf( '%.3d', arbolito), #para que tenga ceros adelante
						'_RunConfig_7', '.csv' )
    fwrite( entrega, 
            file= nom_arch,
            sep= "," )

    cat( arbolito, " " )
  }
  
}